In [1]:
import gdown

In [2]:
file_url="https://drive.google.com/drive/folders/1Kcnsas1hx3t361WxjiULyOdWg1DkoLE9?usp=drive_link"

In [7]:
file_url.split("/")[-1].split("?")[0]

'1Kcnsas1hx3t361WxjiULyOdWg1DkoLE9'

In [3]:
%pwd

'e:\\MLOPS-E2E-Kidney_Classification\\research'

In [1]:
import os,sys

In [2]:
%pwd

'e:\\MLOPS-E2E-Kidney_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
from src.CNN import components,config,constants

In [5]:
from src.CNN.utils.common import read_yaml, create_directories, save_json
from src.CNN.entity.config_entity import PrepareBaseModelConfig
# from src.CNN.config.configuration import ConfigManager
from src.CNN.constants import *

In [6]:
import numpy as np

In [7]:
from src.CNN.entity.config_entity import TrainConfigs

In [8]:
class ModelConfig:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])

    def prepare_data_for_model(self)->TrainConfigs:
        config=self.config.model_train
        
        return TrainConfigs(
            config.model_file_path,
            config.data_file_path,
            config.train_history,
            config.save_weights_path,
            self.params.IMAGE_SIZE,
            self.params.BATCH_SIZE,
            self.params.EPOCHS,
            self.params.CLASSES,
            self.params.featurewise_center,
            self.params.featurewise_std_normalization,
            self.params.rotation_range,
            self.params.width_shift_range,
            self.params.height_shift_range,
            self.params.horizontal_flip,
            self.params.validation_split,
            self.params.rescale,
        )

In [9]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.CNN import logger

In [18]:
from glob import glob
import cv2
import json

In [11]:
from tensorflow.keras.models import load_model
from keras.utils import to_categorical

In [12]:
data=[]
class_name=[]

In [19]:
class TrainModel:
    def __init__(self, config: TrainConfigs):
        self.config = config
        self.data=[]
        self.class_name=[]
        
    
    def train(self):
        self.load_data(self,self.config.data_file_path)
        train,valid=self.preprocess_data(self)
        model=load_model('./artifacts/prepare_base_model/base_model_update.h5')
        logger.info("Model is loaded from {}".format(self.config.model_file_path))
        history=model.fit_generator(
            train,
            epochs=self.config.EPOCHS,
            validation_data=(valid),
        )
        model.save('./'+self.config.save_weights_path+'/weights.h5')
        with open('./'+self.config.train_history, "w") as f:
            json.dump(history.history, f)
            logger.info("Saving history at {}".format(self.config.train_history))
        logger.info("Model is saved at {}".format(self.config.model_file_path))
        # print("Model is saved at {}".format(self.config.model_file_path))
    @staticmethod
    def preprocess_data(self,augmentation=True):
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
            self.data, self.class_name, test_size=0.2, random_state=42
        )
        self.y_train = to_categorical(self.y_train, num_classes=self.config.CLASSES)
        self.y_test = to_categorical(self.y_test, num_classes=self.config.CLASSES)
        # print(self.x_train[0].shape)
        datagen = ImageDataGenerator(
            featurewise_center=self.config.featurewise_center,
            featurewise_std_normalization=self.config.featurewise_std_normalization,
            rotation_range=self.config.rotation_range,
            width_shift_range=self.config.width_shift_range,
            height_shift_range=self.config.height_shift_range,
            horizontal_flip=self.config.horizontal_flip,
            rescale=self.config.rescale,
        )
        validgen = ImageDataGenerator(
            featurewise_center=self.config.featurewise_center,
            featurewise_std_normalization=self.config.featurewise_std_normalization,
            rotation_range=self.config.rotation_range,
            width_shift_range=self.config.width_shift_range,
            height_shift_range=self.config.height_shift_range,
            horizontal_flip=self.config.horizontal_flip,
            rescale=self.config.rescale,
        )
        try:
            datagen.fit(self.x_train)
            validgen.fit(self.x_test)
            # print(dir(train_gen.flow))
            print("Data Augmentation Done")
            logger.info("Data Augmentation Done")
            self.x_train=np.array(self.x_train)
            self.x_test=np.array(self.x_test)
            self.y_train=np.array(self.y_train)
            self.y_test=np.array(self.y_test)
            train_data=datagen.flow(self.x_train,self.y_train)
            valid_data=validgen.flow(self.x_test,self.y_test)
            return train_data,valid_data    
        except Exception as e:
            print(e)
            # logger.info(e)
        # print(x_test_processed[0])
        
    @staticmethod
    def load_data(self,data):
        for index,folder in enumerate(glob("./"+self.config.data_file_path+"/*")):
            for image in glob(folder+"/*.jpg"):
                img = cv2.imread(image)
                img = cv2.resize(img, tuple(self.config.IMAGE_SIZE[:-1]), interpolation=cv2.INTER_AREA)
                img = np.array(img)
                self.data.append(img)
                self.class_name.append(index)
        print("Data Loaded")
        logger.info("Data Loaded")

In [20]:
try:
    model_config = ModelConfig()
    train_config = model_config.prepare_data_for_model()
    train_model = TrainModel(train_config)
    train_model.train()
except Exception as e:
    logger.exception(e)
    raise e

[2023-12-31 13:47:19,028 : INFO : common : Yaml File : config\config.yaml Loaded Successfully : ]
[2023-12-31 13:47:19,031 : INFO : common : Yaml File : params.yaml Loaded Successfully : ]
[2023-12-31 13:47:19,033 : INFO : common : Created Directory : artifacts : ]
Data Loaded
[2023-12-31 13:47:25,849 : INFO : 3190411564 : Data Loaded : ]
Data Augmentation Done
[2023-12-31 13:47:30,122 : INFO : 3190411564 : Data Augmentation Done : ]
[2023-12-31 13:47:31,836 : INFO : 3190411564 : Model is loaded from artifacts/prepare_base_model/base_model_update.h5 : ]


C:\Users\Shreyas\AppData\Local\Temp\ipykernel_82444\3190411564.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(


Epoch 1/20
25/25 [==============================] - 28s 1s/step - loss: 0.4069 - accuracy: 0.9909 - val_loss: 0.4957 - val_accuracy: 0.9845
Epoch 2/20
25/25 [==============================] - 24s 964ms/step - loss: 0.7031 - accuracy: 0.9844 - val_loss: 0.4653 - val_accuracy: 0.9793
Epoch 3/20
25/25 [==============================] - 24s 978ms/step - loss: 1.1819 - accuracy: 0.9896 - val_loss: 1.7804 - val_accuracy: 0.9741
Epoch 4/20
25/25 [==============================] - 24s 969ms/step - loss: 0.7163 - accuracy: 0.9896 - val_loss: 1.9332 - val_accuracy: 0.9534
Epoch 5/20
25/25 [==============================] - 25s 989ms/step - loss: 1.1834 - accuracy: 0.9805 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/20
25/25 [==============================] - 25s 982ms/step - loss: 0.7154 - accuracy: 0.9870 - val_loss: 2.4087 - val_accuracy: 0.9378
Epoch 7/20
25/25 [==============================] - 25s 1000ms/step - loss: 0.2973 - accuracy: 0.9948 - val_loss: 1.0201 - val_accuracy: 0.98

[]